# 4. 分支鏈 (Chains Branching) - Gemini 版本

本範例展示根據條件選擇不同處理路徑，建立一個智能學習助手，根據學生問題類型提供不同的學習建議。

## 學習重點
- 使用 RunnableBranch 實現條件分支
- 學習如何根據輸入內容選擇不同的處理路徑
- 理解分支邏輯的設計模式
- 掌握複雜條件判斷的實現方法


In [ ]:
# 導入必要的套件
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch
from langchain_google_genai import ChatGoogleGenerativeAI

# 載入環境變數
load_dotenv()

# 建立 Gemini 模型
model = ChatGoogleGenerativeAI(model="gemini-flash-2.5")


In [ ]:
# 定義不同學習問題類型的提示模板

# 概念解釋模板
concept_explanation_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的學習導師，擅長用簡單易懂的方式解釋複雜概念。"),
        ("human", "請解釋這個概念：{question}"),
    ]
)

# 解題指導模板
problem_solving_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的學習導師，擅長指導學生解題步驟。"),
        ("human", "請指導我如何解決這個問題：{question}"),
    ]
)

# 學習建議模板
study_advice_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的學習導師，擅長提供學習方法和建議。"),
        ("human", "請給我學習建議：{question}"),
    ]
)

# 複雜問題模板
complex_question_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的學習導師，這個問題比較複雜，需要詳細分析。"),
        ("human", "這是一個複雜問題，請詳細分析：{question}"),
    ]
)

print("學習助手模板已定義：")
print("1. concept_explanation_template: 概念解釋")
print("2. problem_solving_template: 解題指導")
print("3. study_advice_template: 學習建議")
print("4. complex_question_template: 複雜問題")


In [ ]:
# 定義問題分類模板（已移除，改用簡單的字串匹配）
# 原本使用 LLM 分類會造成兩次模型調用，效率較低
# 現在直接在 RunnableBranch 的條件函數中判斷

print("準備建立分支邏輯，使用字串匹配進行問題分類")

In [ ]:
# 定義分支處理邏輯
# 使用 RunnableBranch 根據輸入內容直接選擇不同的處理路徑
# 優化：移除了多餘的分類 LLM 調用，提升效率

branches = RunnableBranch(
    # 概念解釋分支 - 檢查問題中是否包含相關關鍵字
    (
        lambda x: "概念" in x.get("question", "") or "解釋" in x.get("question", "") or "什麼是" in x.get("question", ""),
        concept_explanation_template | model | StrOutputParser()
    ),
    # 解題指導分支
    (
        lambda x: "解題" in x.get("question", "") or "如何" in x.get("question", "") or "怎麼" in x.get("question", "") or "步驟" in x.get("question", ""),
        problem_solving_template | model | StrOutputParser()
    ),
    # 學習建議分支
    (
        lambda x: "學習" in x.get("question", "") or "方法" in x.get("question", "") or "建議" in x.get("question", "") or "技巧" in x.get("question", ""),
        study_advice_template | model | StrOutputParser()
    ),
    # 預設分支（複雜問題）
    complex_question_template | model | StrOutputParser()
)

print("分支處理邏輯已定義（優化版）：")
print("1. 概念問題 → 概念解釋")
print("2. 解題問題 → 解題指導")
print("3. 學習問題 → 學習建議")
print("4. 其他情況 → 複雜問題分析")
print("\n✨ 優化：直接使用字串匹配，避免多餘的 LLM 調用，提升效率！")

In [ ]:
# 簡化後的鏈：直接使用分支邏輯
# 移除了分類鏈，效率提升約 50%（減少一次 LLM 調用）
chain = branches

print("完整的學習助手鏈已建立（優化版）！")
print("鏈的流程：學習問題 → 字串匹配分類 → 選擇處理路徑 → 生成學習建議")
print("\n💡 效能提升：")
print("  - 原版：需要 2 次 LLM 調用（分類 + 回答）")
print("  - 優化版：只需 1 次 LLM 調用（直接回答）")

In [ ]:
# 測試不同類型的學習問題
test_questions = [
    "什麼是機器學習？",  # 概念解釋
    "如何解二元一次方程式？",  # 解題指導
    "有什麼好的學習方法？",  # 學習建議
    "請分析這個複雜的數學問題..."  # 複雜問題
]

print("=" * 60)
print("智能學習助手測試結果：")
print("=" * 60)

for i, question in enumerate(test_questions, 1):
    print(f"\n測試 {i}: {question}")
    print("-" * 40)
    result = chain.invoke({"question": question})
    print(f"學習建議: {result}")
    print("=" * 60)


## 💡 重點說明

### 分支鏈的設計模式

1. **條件判斷**: 使用 lambda 函數檢查輸入內容
2. **分支處理**: 每個分支都有專門的處理邏輯
3. **預設處理**: 提供預設分支處理未分類的情況

### RunnableBranch 的使用

```python
branches = RunnableBranch(
    (條件函數1, 處理鏈1),
    (條件函數2, 處理鏈2),
    (條件函數3, 處理鏈3),
    預設處理鏈  # 最後一個是預設分支
)
```

### 條件函數的設計

```python
# 條件函數應該返回 True 或 False
# 使用 x.get("question", "") 安全地取值，避免 KeyError
lambda x: "概念" in x.get("question", "") or "解釋" in x.get("question", "")
```

### ⚡ 效能優化重點

**優化前的問題**:
```python
# ❌ 效率低：需要兩次 LLM 調用
chain = classification_chain | branches
# 第1次：classification_chain 呼叫 LLM 進行分類
# 第2次：branches 中的對應鏈再次呼叫 LLM
```

**優化後的方案**:
```python
# ✅ 效率高：只需一次 LLM 調用
chain = branches
# 使用字串匹配直接分類，然後調用對應的 LLM
```

**效能提升**:
- 減少 50% 的 LLM 調用次數
- 降低約 50% 的回應時間
- 節省 API 使用成本

### 實際應用場景

- **智能學習助手**: 根據問題類型提供不同指導
- **客服系統**: 自動分類客戶問題
- **內容推薦**: 根據用戶偏好推薦內容
- **智能路由**: 根據請求類型選擇處理方式